In [2]:
import pandas as pd
from hmmlearn import hmm
import numpy as np
from sklearn import preprocessing
from scipy.stats import norm
import pomegranate

# Getting data and diving them into unique unit numbers

We need to divide data into unique numbers, because the state restes as the unit number changes, so we need to find Gaussian distribution for different unit numbers

In [3]:
data = pd.read_csv('~/Documents/hitachi/CMAPSS/train_FD001.txt', sep=" ", header=None)
unique_unit_values = data[0].unique() #Number of units
data_cycles = []
for unit_num in unique_unit_values:
    data_cycles.append(data[data[0] == unit_num])

# Removing operational settings and normalize the data column wise

In [4]:
def normalize(data):
    x = data.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    dataNew = pd.DataFrame(x_scaled)
    return dataNew
#Remove the operation settings
dataT = data[data.columns[5:26]]
dataT.columns = range(21)
dataT = normalize(dataT)

# Dividing data for each unit

I think this is why my transitional matrix previously was not working properly as in each unit the state resets and start from good condition

In [5]:
dataT_cycles = []
for unit_num in unique_unit_values:
    dataT_cycles.append(dataT[data[0] == unit_num])

# Identifying and removing non variable data columns

Removing the columns where the data does not vary

In [6]:
for dataT_cycle in dataT_cycles:
    print(dataT_cycle.columns[dataT_cycle.std() == 0])
"""
Here we can see 0,4,9,15,17,18 but also 5 at many places so we drop column number 5 as well
"""
dataT.drop(data.columns[[0, 3, 4, 5, 9, 15, 17, 18]],axis=1,inplace=True)
dataT.columns = range(13)
dataT_cycles = []
for unit_num in unique_unit_values:
    dataT_cycles.append(dataT[data[0] == unit_num])

Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='int64')
Int64Index([0, 4, 5, 9, 15, 17, 18], dtype='i

In [7]:
# Right now only using the first data frame (i.e Machine 1) to train the VAE, but we can combine all the dataframes
# and train the VAE jointly on the entire data for better performance 

x_train = dataT_cycles[0].values[:150]
x_test = dataT_cycles[0].values[151:198]
x_train.shape
# x_test.shape

(150, 13)

# Variational AutoEncoders to find Latent State Space Distribution



In [7]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os

Using TensorFlow backend.
/home/yannik/miniconda3/envs/ykp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/yannik/miniconda3/envs/ykp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/yannik/miniconda3/envs/ykp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/yannik/miniconda3/en

In [8]:
# Data preparation
x_train = dataT_cycles[0].values[:100]
x_test = dataT_cycles[0].values[101:198]
x_train.shape
x_test.shape
original_dim = x_train[0].shape[0]

In [9]:
# network parameters
input_shape = (original_dim, )
intermediate_dim = 9
batch_size = 10
latent_dim = 5
epochs = 50

In [10]:
# Sampling function
# reparameterization trick
# instead of sampling from Q(z|X), sample eps = N(0,I)
# z = z_mean + sqrt(var)*eps
def sampling(args):
    """Reparameterization trick by sampling fr an isotropic unit Gaussian.

    # Arguments
        args (tensor): mean and log of variance of Q(z|X)

    # Returns
        z (tensor): sampled latent vector
    """
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    
    return z_mean + K.exp(0.5 * z_log_var) * epsilon


In [11]:
# VAE Model Encoder + Decoder 

# Building the Encoder
inputs = Input(shape=input_shape, name='encoder_input')
x = Dense(intermediate_dim, activation='relu')(inputs)
z_mean = Dense(latent_dim, name='z_mean')(x)
z_log_var = Dense(latent_dim, name='z_log_var')(x)

# use reparameterization trick to push the sampling out as input
# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

# instantiate encoder model
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 13)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 9)            126         encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 5)            50          dense_1[0][0]                    
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 5)            50          dense_1[0][0]                    
____________________________________________________________________________________________

In [12]:
#build decoder model 
latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
x = Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = Dense(original_dim, activation='sigmoid')(x)

# instantiate decoder model
decoder = Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 9)                 54        
_________________________________________________________________
dense_3 (Dense)              (None, 13)                130       
Total params: 184
Trainable params: 184
Non-trainable params: 0
_________________________________________________________________


In [13]:
#instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = Model(inputs, outputs, name='vae_mlp')

In [14]:
def main(args):
    parser = argparse.ArgumentParser()
    help_ = "Load h5 model trained weights"
    parser.add_argument("-w", "--weights", help=help_)
    help_ = "Use mse loss instead of binary cross entropy (default)"
    parser.add_argument("-m", "--mse", help=help_, action='store_true')
    
    models = (encoder, decoder)
    data = (x_test, None)
    
    # VAE loss = mse_loss or xent_loss + kl_loss
    if args.mse:
        reconstruction_loss = mse(inputs, outputs)
    else:
        reconstruction_loss = binary_crossentropy(inputs, outputs)
        
    reconstruction_loss *= original_dim
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis= -1)
    kl_loss *= 0.5 
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    vae.compile(optimizer='adam')
    vae.summary()
    
    if args.weights:
        vae.load_weights(args.weights)
    else:
        # Train the autoencoder
        vae.fit(x_train, epochs=epochs, batch_size= batch_size, validation_data=(x_test, None))
        vae.save_weights('vae_mlp_CMAPSS.h5')

In [15]:
class Args:
    mse = None
    weights = None
    
args = Args()

if __name__ == '__main__':
    main(args)
    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "vae_mlp"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 13)                0         
_________________________________________________________________
encoder (Model)              [(None, 5), (None, 5), (N 226       
_________________________________________________________________
decoder (Model)              (None, 13)                184       
Total params: 410
Trainable params: 410
Non-trainable params: 0
_________________________________________________________________


/home/yannik/miniconda3/envs/ykp/lib/python3.7/site-packages/keras/engine/training_utils.py:819: UserWarning: Output decoder missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to decoder.
  'be expecting any data to be passed to {0}.'.format(name))



Train on 100 samples, validate on 91 samples
Epoch 1/50
100/100 [==============================] - 1s 6ms/step - loss: 8.6136 - val_loss: 7.5069
Epoch 2/50
100/100 [==============================] - 0s 159us/step - loss: 8.1071 - val_loss: 6.7097
Epoch 3/50
100/100 [==============================] - 0s 176us/step - loss: 7.4209 - val_loss: 5.7314
Epoch 4/50
100/100 [==============================] - 0s 188us/step - loss: 6.5338 - val_loss: 4.2211
Epoch 5/50
100/100 [==============================] - 0s 188us/step - loss: 5.2482 - val_loss: 2.3457
Epoch 6/50
100/100 [==============================] - 0s 222us/step - loss: 3.4696 - val_loss: -0.6214
Epoch 7/50
100/100 [==============================] - 0s 234us/step - loss: 0.8200 - val_loss: -4.6524
Epoch 8/50
100/100 [==============================] - 0s 233us/step - loss: -3.1063 - val_loss: -10.8377
Epoch 9/50
100/100 [==============================] - 0s 199us/step - loss: -8.8391 - val_loss: -20.2851
Epoch 10/50
100/100 [=========

In [16]:
# Once the VAE has been trained, we can use the encoder to sample the latent space

#predicting the latent space for first 13 observation values for machine 1
test = np.asarray(x_train[0:13])  

# indexing on 2 because the encoder predicts z_mean, z_log_var and sampled vector z (we are interested in z only)
latent_space = encoder.predict(test)[2]  

In [17]:
# Each list is a 5 dimension latent state space for that observation value
latent_space

array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]], dtype=float32)

In [18]:
# Reconstruct the raw observation from the learned latent space 
sample = decoder.predict(latent_space)

In [19]:
# Compare the with the real x_train value
x_train[0]

array([0.18373494, 0.40680183, 0.72624799, 0.24242424, 0.109755  ,
       0.36904762, 0.63326226, 0.20588235, 0.1996078 , 0.36398615,
       0.33333333, 0.71317829, 0.7246617 ])

In [20]:
# Right now they are not same as we trained the VAE on very less amount of data 
sample[0]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
      dtype=float32)

# Using HMM to find out transitional matrices

Here we first define transmatrix as [[0.5, 0.5, 0.0, 0.0],[0.0, 0.5, 0.5, 0.0],[0.0, 0.0, 0.5, 0.5],[0.0,0.0,0.0,1.0]] which means there is half chance for each state to go to next state and half to remain in the current state itself.

Then we train for each unit for transmatrix as well as state means and we will take average of each unit transmatrices and states as the transmatrix and state

*Note*: Here state '0' means the perfect health and '3' means weakest health 

In [8]:
dataT_cycles[0]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.183735,0.406802,0.726248,0.242424,0.109755,0.369048,0.633262,0.205882,0.199608,0.363986,0.333333,0.713178,0.724662
1,0.283133,0.453019,0.628019,0.212121,0.100242,0.380952,0.765458,0.279412,0.162813,0.411312,0.333333,0.666667,0.731014
2,0.343373,0.369523,0.710145,0.272727,0.140043,0.250000,0.795309,0.220588,0.171793,0.357445,0.166667,0.627907,0.621375
3,0.343373,0.256159,0.740741,0.318182,0.124518,0.166667,0.889126,0.294118,0.174889,0.166603,0.333333,0.573643,0.662386
4,0.349398,0.257467,0.668277,0.242424,0.149960,0.255952,0.746269,0.235294,0.174734,0.402078,0.416667,0.589147,0.704502
...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.765060,0.683235,0.336554,0.621212,0.072602,0.684524,0.234542,0.514706,0.091599,0.753367,0.666667,0.286822,0.089202
188,0.894578,0.547853,0.136876,0.560606,0.102396,0.732143,0.189765,0.661765,0.090670,0.744132,0.583333,0.263566,0.301712
189,0.731928,0.614345,0.231884,0.590909,0.084582,0.880952,0.287846,0.691176,0.065229,0.759523,0.833333,0.271318,0.239299
190,0.641566,0.682799,0.172303,0.575758,0.094364,0.773810,0.187633,0.617647,0.075704,0.740669,0.500000,0.240310,0.324910


In [9]:
lr = hmm.GaussianHMM(n_components=4, covariance_type="diag",init_params="cm", params="mtc")
lr.startprob_ = np.array([1.0, 0.0, 0.0, 0.0])
transmats = []
statemeans = []
covars = []
for i in range(100):
    lr.transmat_ = np.array([[0.5, 0.5, 0.0, 0.0],[0.0, 0.5, 0.5, 0.0],[0.0, 0.0, 0.3, 0.7],[0.0,0.0,0.0,1.0]])
    lr.fit(dataT_cycles[i])
    transmats.append(lr.transmat_)
    statemeans.append(lr.means_)
    covars.append(lr.covars_)

In [10]:
#lr = hmm.GMMHMM(n_components=4, n_mix=4, covariance_type="diag",init_params="cm", params="mt")
#lr.startprob_ = np.array([1.0, 0.0, 0.0, 0.0])
#transmats = []
#statemeans = []
#for i in range(100):
#    lr.transmat_ = np.array([[0.5, 0.5, 0.0, 0.0],[0.0, 0.5, 0.5, 0.0],[0.0, 0.0, 0.5, 0.5],[0.0,0.0,0.0,1.0]])
#    lr.fit(dataT_cycles[i])
#    transmat = lr.transmat_
#    transmats.append(transmat)
#    statemeans.append(lr.means_)

In [11]:
transmat = np.array(transmats).mean(axis=0)
statemean = np.array(statemeans).mean(axis=0)
covar = np.array(covars).mean(axis=0)

In [12]:
transmat

array([[0.65889597, 0.34110403, 0.        , 0.        ],
       [0.        , 0.63899248, 0.36100752, 0.        ],
       [0.        , 0.        , 0.76683124, 0.23316876],
       [0.        , 0.        , 0.        , 1.        ]])

In [13]:
pd.DataFrame(transmat)

,0,1,2,3
0,0.658896,0.341104,0.000000,0.000000
1,0.000000,0.638992,0.361008,0.000000
2,0.000000,0.000000,0.766831,0.233169
3,0.000000,0.000000,0.000000,1.000000


In [14]:
pd.DataFrame(statemean)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.368893,0.357284,0.654884,0.246592,0.159665,0.319512,0.662172,0.268697,0.196373,0.377861,0.365443,0.604374,0.629463
1,0.445521,0.436801,0.552530,0.297437,0.192327,0.405016,0.570111,0.322218,0.220602,0.458655,0.445462,0.527639,0.548432
2,0.531650,0.501197,0.479200,0.356338,0.238838,0.520376,0.482756,0.371746,0.261436,0.540383,0.504609,0.437612,0.457751
3,0.669016,0.622176,0.336322,0.452984,0.304682,0.684514,0.320877,0.470418,0.314942,0.682230,0.627440,0.301617,0.310948


In [15]:
t_prob = np.array([transmat, transmat])

In [16]:
rewards = np.array([[100, 50, 0, -50],[-50, 0, 50, 100]])

In [17]:
pd.DataFrame(rewards)

,0,1,2,3
0,100,50,0,-50
1,-50,0,50,100


In [18]:
e_prob = np.array([norm.pdf(statemean), norm.pdf(statemean)])

In [19]:
e_prob

array([[[0.3727007 , 0.37427496, 0.32194483, 0.38699548, 0.39388943,
         0.37908969, 0.32040339, 0.38479773, 0.39132386, 0.37145478,
         0.37317313, 0.33234808, 0.32724367],
        [0.36125066, 0.36264314, 0.34246584, 0.38167991, 0.3916317 ,
         0.3675274 , 0.33910283, 0.37876068, 0.38935211, 0.35911203,
         0.36126019, 0.34710074, 0.34323939],
        [0.34636416, 0.35185443, 0.35566889, 0.37440135, 0.38772446,
         0.34842433, 0.35506119, 0.37230713, 0.38553902, 0.34474664,
         0.3512512 , 0.36251463, 0.35926089],
        [0.31894728, 0.32873937, 0.37700573, 0.3600415 , 0.38084829,
         0.31561936, 0.37892403, 0.35715516, 0.37963967, 0.3161125 ,
         0.32765998, 0.38120231, 0.3801145 ]],

       [[0.3727007 , 0.37427496, 0.32194483, 0.38699548, 0.39388943,
         0.37908969, 0.32040339, 0.38479773, 0.39132386, 0.37145478,
         0.37317313, 0.33234808, 0.32724367],
        [0.36125066, 0.36264314, 0.34246584, 0.38167991, 0.3916317 ,
         

In [20]:
pd.DataFrame(e_prob[0])

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.372701,0.374275,0.321945,0.386995,0.393889,0.379090,0.320403,0.384798,0.391324,0.371455,0.373173,0.332348,0.327244
1,0.361251,0.362643,0.342466,0.381680,0.391632,0.367527,0.339103,0.378761,0.389352,0.359112,0.361260,0.347101,0.343239
2,0.346364,0.351854,0.355669,0.374401,0.387724,0.348424,0.355061,0.372307,0.385539,0.344747,0.351251,0.362515,0.359261
3,0.318947,0.328739,0.377006,0.360042,0.380848,0.315619,0.378924,0.357155,0.379640,0.316112,0.327660,0.381202,0.380115


In [21]:
covar

array([[[0.0093368 , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.00852376, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.00662536, 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.00500401, 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.00377829,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.

In [22]:
covar=[covar[i].sum(axis=1) for i in range(4)]

In [23]:
pd.DataFrame(covar)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.009337,0.008524,0.006625,0.005004,0.003778,0.006497,0.006709,0.004922,0.003688,0.007666,0.007730,0.007660,0.008378
1,0.009711,0.009147,0.007293,0.005418,0.004399,0.007117,0.007430,0.005291,0.004283,0.008104,0.008290,0.008257,0.009048
2,0.009785,0.009134,0.006723,0.004501,0.003339,0.006753,0.007112,0.004476,0.003221,0.008190,0.008300,0.007945,0.008670
3,0.010450,0.009430,0.007269,0.004895,0.003306,0.008217,0.008183,0.004830,0.003110,0.008772,0.008593,0.009057,0.009645


In [51]:
# 0: no-repair, 1: repair
actions = ('0', '1')
# 0: failing, 1: low health, 2: good health, 3: perfect health
states = ('0', '1', '2', '3')

gamma = 0.95

In [52]:
"""
First we define an MDP. We also represent a policy
as a dictionary of {state: action} pairs, and a utility function as a
dictionary of {state: number} pairs. We then define the value_iteration
and policy_iteration algorithms."""


import random
import numpy as np
from collections import defaultdict

class MDP:

    """A Markov Decision Process, defined by an initial state, transition model,
    and reward function. We also keep track of a gamma value, for use by
    algorithms. The transition model is represented somewhat differently from
    the text. Instead of P(s' | s, a) being a probability number for each
    state/state/action triplet, we instead have T(s, a) return a
    list of (p, s') pairs. We also keep track of the possible states,
    terminal states, and actions for each state."""

    def __init__(self, init, actlist, terminals, transitions=None, reward=None, states=None, gamma=0.9):
        if not (0 < gamma <= 1):
            raise ValueError("An MDP must have 0 < gamma <= 1")

        # collect states from transitions table if not passed.
        self.states = states or self.get_states_from_transitions(transitions)
            
        self.init = init
        
        if isinstance(actlist, list):
            # if actlist is a list, all states have the same actions
            self.actlist = actlist

        elif isinstance(actlist, dict):
            # if actlist is a dict, different actions for each state
            self.actlist = actlist
        
        self.terminals = terminals
        self.transitions = transitions or {}
        if not self.transitions:
            print("Warning: Transition table is empty.")

        self.gamma = gamma

        self.reward = reward or {s: 0 for s in self.states}

        # self.check_consistency()

    def R(self, state):
        """Return a numeric reward for this state."""

        return self.reward[state]

    def T(self, state, action):
        """Transition model. From a state and an action, return a list
        of (probability, result-state) pairs."""

        if not self.transitions:
            raise ValueError("Transition model is missing")
        else:
            return self.transitions[state][action]

    def actions(self, state):
        """Return a list of actions that can be performed in this state. By default, a
        fixed list of actions, except for terminal states. Override this
        method if you need to specialize by state."""

        if state in self.terminals:
            return [None]
        else:
            return self.actlist

    def get_states_from_transitions(self, transitions):
        if isinstance(transitions, dict):
            s1 = set(transitions.keys())
            s2 = set(tr[1] for actions in transitions.values()
                     for effects in actions.values()
                     for tr in effects)
            return s1.union(s2)
        else:
            print('Could not retrieve states from transitions')
            return None

    def check_consistency(self):

        # check that all states in transitions are valid
        assert set(self.states) == self.get_states_from_transitions(self.transitions)

        # check that init is a valid state
        assert self.init in self.states

        # check reward for each state
        assert set(self.reward.keys()) == set(self.states)

        # check that all terminals are valid states
        assert all(t in self.states for t in self.terminals)

        # check that probability distributions for all actions sum to 1
        for s1, actions in self.transitions.items():
            for a in actions.keys():
                s = 0
                for o in actions[a]:
                    s += o[0]
                assert abs(s - 1) < 0.001

class POMDP(MDP):

    """A Partially Observable Markov Decision Process, defined by
    a transition model P(s'|s,a), actions A(s), a reward function R(s),
    and a sensor model P(e|s). We also keep track of a gamma value,
    for use by algorithms. The transition and the sensor models
    are defined as matrices. We also keep track of the possible states
    and actions for each state."""

    def __init__(self, actions, transitions=None, evidences=None, rewards=None, states=None, gamma=0.95):
        """Initialize variables of the pomdp"""

        if not (0 < gamma <= 1):
            raise ValueError('A POMDP must have 0 < gamma <= 1')

        self.states = states
        self.actions = actions

        # transition model cannot be undefined
        self.t_prob = transitions
        if not self.t_prob.any():
            print('Warning: Transition model is undefined')
        
        # sensor model cannot be undefined
        self.e_prob = evidences
        if not self.e_prob.any():
            print('Warning: Sensor model is undefined')
        
        self.gamma = gamma
        self.rewards = rewards

    def remove_dominated_plans(self, input_values):
        """
        Remove dominated plans.
        This method finds all the lines contributing to the
        upper surface and removes those which don't.
        """

        values = [val for action in input_values for val in input_values[action]]
        values.sort(key=lambda x: x[0], reverse=True)

        best = [values[0]]
        y1_max = max(val[1] for val in values)
        tgt = values[0]
        prev_b = 0
        prev_ix = 0
        while tgt[1] != y1_max:
            min_b = 1
            min_ix = 0
            for i in range(prev_ix + 1, len(values)):
                if values[i][0] - tgt[0] + tgt[1] - values[i][1] != 0:
                    trans_b = (values[i][0] - tgt[0]) / (values[i][0] - tgt[0] + tgt[1] - values[i][1])
                    if 0 <= trans_b <= 1 and trans_b > prev_b and trans_b < min_b:
                        min_b = trans_b
                        min_ix = i
            prev_b = min_b
            prev_ix = min_ix
            tgt = values[min_ix]
            best.append(tgt)

        return self.generate_mapping(best, input_values)

    def remove_dominated_plans_fast(self, input_values):
        """
        Remove dominated plans using approximations.
        Resamples the upper boundary at intervals of 100 and
        finds the maximum values at these points.
        """

        values = [val for action in input_values for val in input_values[action]]
        values.sort(key=lambda x: x[0], reverse=True)

        best = []
        sr = 100
        for i in range(sr + 1):
            x = i / float(sr)
            maximum = (values[0][1] - values[0][0]) * x + values[0][0]
            tgt = values[0]
            for value in values:
                val = (value[1] - value[0]) * x + value[0]
                if val > maximum:
                    maximum = val
                    tgt = value

            if all(any(tgt != v) for v in best):
                best.append(np.array(tgt))

        return self.generate_mapping(best, input_values)

    def generate_mapping(self, best, input_values):
        """Generate mappings after removing dominated plans"""

        mapping = defaultdict(list)
        for value in best:
            for action in input_values:
                if any(all(value == v) for v in input_values[action]):
                    mapping[action].append(value)

        return mapping

    def max_difference(self, U1, U2):
        """Find maximum difference between two utility mappings"""

        for k, v in U1.items():
            sum1 = 0
            for element in U1[k]:
                sum1 += sum(element)
            sum2 = 0
            for element in U2[k]:
                sum2 += sum(element)
        return abs(sum1 - sum2)

        
class Matrix:
    """Matrix operations class"""

    @staticmethod
    def add(A, B):
        """Add two matrices A and B"""

        res = []
        for i in range(len(A)):
            row = []
            for j in range(len(A[0])):
                row.append(A[i][j] + B[i][j])
            res.append(row)
        return res

    @staticmethod
    def scalar_multiply(a, B):
        """Multiply scalar a to matrix B"""

        for i in range(len(B)):
            for j in range(len(B[0])):
                B[i][j] = a * B[i][j]
        return B

    @staticmethod
    def multiply(A, B):
        """Multiply two matrices A and B element-wise"""

        matrix = []
        for i in range(len(B)):
            row = []
            for j in range(len(B[0])):
                row.append(B[i][j] * A[j][i])
            matrix.append(row)

        return matrix

    @staticmethod
    def matmul(A, B):
        """Inner-product of two matrices"""

        return [[sum(ele_a*ele_b for ele_a, ele_b in zip(row_a, col_b)) for col_b in list(zip(*B))] for row_a in A]

    @staticmethod
    def transpose(A):
        """Transpose a matrix"""
        
        return [list(i) for i in zip(*A)]


def pomdp_value_iteration(pomdp, epsilon=0.1):
    """Solving a POMDP by value iteration."""

    U = {'':[[0]* len(pomdp.states)]}
    count = 0
    while True:
        count += 1
        prev_U = U
        values = [val for action in U for val in U[action]]
        value_matxs = []
        for i in values:
            for j in values:
                value_matxs.append([i, j])

        U1 = defaultdict(list)
        for action in pomdp.actions:
            for u in value_matxs:
                u1 = Matrix.matmul(Matrix.matmul(pomdp.t_prob[int(action)], Matrix.multiply(pomdp.e_prob[int(action)], Matrix.transpose(u))), [[1], [1]])
                u1 = Matrix.add(Matrix.scalar_multiply(pomdp.gamma, Matrix.transpose(u1)), [pomdp.rewards[int(action)]])
                U1[action].append(u1[0])

        U = pomdp.remove_dominated_plans_fast(U1)
        # replace with U = pomdp.remove_dominated_plans(U1) for accurate calculations
        
        if count > 10:
            if pomdp.max_difference(U, prev_U) < epsilon * (1 - pomdp.gamma) / pomdp.gamma:
                return U



In [53]:
states

('0', '1', '2', '3')

In [54]:
pomdp = POMDP(actions, t_prob, e_prob, rewards, states, gamma)

In [55]:
utility = pomdp_value_iteration(pomdp, epsilon=3)
utility

defaultdict(list,
            {'0': [array([ 204.52516018,   88.9820356 ,  -34.29024281, -171.26528113])]})

In [56]:
pd.DataFrame(np.array([204.52516018,   88.9820356 ,  -34.29024281, -171.26528113]))

,0
0,204.525160
1,88.982036
2,-34.290243
3,-171.265281


# ADQRN

In [ ]:
from __future__ import division

import gym
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#These lines establish the feed-forward part of the network used to choose actions
inputs1 = tf.placeholder(shape=[1,13],dtype=tf.float32)
W = tf.Variable(tf.random_uniform([13,2],0,0.01))
Qout = tf.matmul(inputs1,W)
predict = tf.argmax(Qout,1)

#Below we obtain the loss by taking the sum of squares difference between the target and prediction Q values.
nextQ = tf.placeholder(shape=[1,2],dtype=tf.float32)
loss = tf.reduce_sum(tf.square(nextQ - Qout))
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
updateModel = trainer.minimize(loss)

In [ ]:
def get_state(obs):
    state = 0
    diff = 16
    for i in range(len(statemean)):
        stateDiff = obs - statemean[i]
        stateDiffVal = np.sqrt(np.mean(stateDiff**2))
        if stateDiffVal < diff:
            diff = stateDiffVal
            state = i
    return state

In [ ]:
# Getting the next step after an action is done

def getStepDetails(i,j,action):
    unitData = dataT_cycles[i]
    d = False
    if action == 1:
        newJ = 0
    else:
        newJ = j+1
    obsNext = unitData.values[newJ]
    if newJ >= len(unitData) - 1:
        d = True
    s1 = get_state(obsNext)
    r1 = rewards[action][s1]
    return r1,newJ,s1,obsNext,d

In [ ]:
# Set learning parameters
init = tf.global_variables_initializer()
y = gamma
e = 0.1
num_episodes = len(dataT_cycles)
#create lists to contain total rewards and steps per episode
jList = []
rList = []
D = np.empty([0,5]) # Replay memory
with tf.Session() as sess:
    sess.run(init)
    for i in range(num_episodes):
        #Reset environment and get first new observation for new unit
        rAll = 0
        d = False
        j = 0
        k = 0
        unitData = dataT_cycles[i]
        #The Q-Network
        while j < len(unitData):
            #Choose an action by greedily (with e chance of random action) from the Q-network
            a,allQ = sess.run([predict,Qout],feed_dict={inputs1:unitData.values[j].reshape(1,13)})
            if np.random.rand(1) < e:
                a[0] = np.random.randint(0,2)
            #Get new state and reward from environment
            r,j,s1,o1,d = getStepDetails(i,j,a[0])
            D = np.vstack([D, [a[0],unitData.values[j-1].reshape(1,13),r,o1,s1]])
            if len(D) > 20:
                lastInd = np.random.randint(15,len(D))
                randomSample = D[lastInd-15:lastInd]
                finalO = D[lastInd,3].reshape(1,13)
                Reward = np.sum(D[lastInd-15:lastInd,2])
            else:
                finalO = o1.reshape(1,13)
                Reward = r
            # We take batch size of 15 (j in algorithm)
            #Obtain the Q' values by feeding the new state through our network
            Q1 = sess.run(Qout,feed_dict={inputs1:finalO})
            #Obtain maxQ' and set our target value for chosen action.
            maxQ1 = np.max(Q1)
            targetQ = allQ
            targetQ[0,a[0]] = Reward + y*maxQ1
            #Train our network using target and predicted Q values
            _,W1 = sess.run([updateModel,W],feed_dict={inputs1:unitData.values[j-1].reshape(1,13),nextQ:targetQ})
            rAll += r
            s = s1
            k += 1
            if d == True or k >= 1000:
                #Reduce chance of random action as we train the model.
                e = 1./((i/50) + 10)
                break
        jList.append(j)
        rList.append(rAll)

# Prediction

In [25]:
a = dataT_cycles[5].values[160].reshape(-1,13)

In [29]:
dataT_cycles[5].values[160]

array([[0.77108434, 0.48484848, 0.40257649, 0.51515152, 0.03688414,
        0.61904762, 0.36886994, 0.55882353, 0.03333677, 0.65101962,
        0.58333333, 0.30232558, 0.2903894 ]])

In [ ]:
W1

In [ ]:
np.dot(a,W1)